In [1]:
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as pltt

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
x_mean = np.mean(x_train, axis=(0, 1, 2))
x_std = np.std(x_train, axis=(0, 1, 2))

In [4]:
x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=1)

In [6]:
# transfer learning (전이학습) : pre-trained (사전학습) 되어 있는 모델을 가지고 와서 사용하고 싶다.
# VGG 16 : Visual Geometry Group 16ro layer -  이전에 대용량의 이미지를 이미 학습한 모델이다.
# weights="imagenet" :  imagenet datasets 로 학습된 가중치를 사용하겠다. (pre-trained)
# include_top=False : fc layer를 사용하지 않겠다. 풀리 커넥티드 레이어
vgg16 = VGG16(input_shape=(32, 32, 3), weights="imagenet", include_top=False)

In [7]:
# 13개의 convolution layer + 3개의 fc layer
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# 마지막 4개의 layer를 제외하고, 나머지 layer들은 학습하지 않겠다 (= w, b 변경하지 않겠다)
# 얘네들이 가지고 있는 특성들을 그대로 사용하지만 마지막 4개만 우리의 특징에 맞게 변경하여서 사용하겠다. 모델의 능력을 최대한 사용하기 위해서
for layer in vgg16.layers[:-4]:
    layer.trainable = False

In [9]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
# BatchNormalization : layer의 입력을 평균이 0, 분산이 1 이 되도록 정규화 -> 데이터의 입력 분포가 일관되게 유지 = 학습 속도가 빨라짐(과적합도 낮춰주기도 한다)
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(10, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,849,610 (56.65 MB)

 Trainable params: 7,213,834 (27.52 MB)

 Non-trainable params: 7,635,776 (29.13 MB)

In [11]:
# Adam(0.001) : learning 레이트 를 지정해줌
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(0.001), metrics=["acc"])

In [12]:
# fine tuning : 이미 학습되어 있는 모델을, 우리의 데이터에 맞게 w, b 값을 미세하게 조정 (학습)
result = model.fit(x_train, y_train, epochs=10, callbacks=[EarlyStopping(patience=3)], validation_data=(x_val, y_val))

Epoch 1/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 207s 189ms/step - acc: 0.6228 - loss: 1.0866 - val_acc: 0.6497 - val_loss: 1.0285
Epoch 2/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 200s 183ms/step - acc: 0.7455 - loss: 0.7360 - val_acc: 0.7551 - val_loss: 0.7093
Epoch 3/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 201s 184ms/step - acc: 0.7814 - loss: 0.6321 - val_acc: 0.7510 - val_loss: 0.7477
Epoch 4/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 200s 183ms/step - acc: 0.8096 - loss: 0.5433 - val_acc: 0.7257 - val_loss: 0.8351
Epoch 5/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 203s 186ms/step - acc: 0.8415 - loss: 0.4523 - val_acc: 0.7629 - val_loss: 0.6968
Epoch 6/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 209s 191ms/step - acc: 0.8688 - loss: 0.3749 - val_acc: 0.7641 - val_loss: 0.7658
Epoch 7/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 203s 186ms/step - acc: 0.8970 - loss: 0.3014 - val_acc: 0.7672 - val_loss: 0.7860
Epoch 8/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 206s 188ms/step - acc: 0.9178 - loss: 0.2345 - val_acc: 0.7660 - val_loss: 0.8463


In [13]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 74ms/step - acc: 0.7610 - loss: 0.8888


[0.8699984550476074, 0.7616999745368958]

In [ ]:
# 학습할 때 4개 뺀것도 변경해보고, 히든레이어도 추가하면서 가장 적합한 과정을 찾아가야 한다.